In [45]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import onlypores as op
import openpyxl
from pathlib import Path

In [46]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [47]:
wb = openpyxl.load_workbook(filename = r'Y:\Database\Database.xlsx')

sheet = wb.active

In [48]:
ids = [s.value for s in sheet['A'][1:]]

paths  = [s.value for s in sheet['E'][1:]]

paths_lentes = [s.value for s in sheet['S'][1:]]

onlypores = [s.value for s in sheet['H'][1:]]

masks = [s.value for s in sheet['G'][1:]]

segmenteds = [s.value for s in sheet['F'][1:]]

onlypores_lentes = [s.value for s in sheet['V'][1:]]

masks_lentes = [s.value for s in sheet['U'][1:]]

segmenteds_lentes = [s.value for s in sheet['T'][1:]]

In [49]:
for i in range(len(ids)):

    print(f'Processing {ids[i]}')

    if (paths[i] != None) and ((onlypores[i] is None) or (masks[i] is None) or (segmenteds[i] is None)):

        try:

            print(f'Processing {paths[i]}')

            xct_folder = Path(paths[i])
            volume = read_sequence(xct_folder)
            onlypores, sample_mask, binary = op.onlypores(volume)

            output_folder = xct_folder.parent / 'segmentation'

            #save the segmented volume
            write_sequence(output_folder, 'onlypores', onlypores.astype(np.uint8)*255)
            #save the sample mask
            write_sequence(output_folder, 'sample_mask', sample_mask.astype(np.uint8)*255)
            #save the segmented volume
            write_sequence(output_folder, 'binary', binary.astype(np.uint8)*255)

            sheet[f'H{i+2}'] = str(output_folder / 'onlypores')
            sheet[f'G{i+2}'] = str(output_folder / 'sample_mask')
            sheet[f'F{i+2}'] = str(output_folder / 'binary')
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)
    
    if (paths_lentes[i] != None) and ((onlypores_lentes[i] is None) or (masks_lentes[i] is None) or (segmenteds_lentes[i] is None)):

        try:

            print(f'Processing {paths_lentes[i]}')

            xct_folder = Path(paths_lentes[i])
            volume = read_sequence(xct_folder)
            onlypores, sample_mask, binary = op.onlypores(volume)

            output_folder = xct_folder.parent / 'segmentation'

            #save the segmented volume
            write_sequence(output_folder, 'onlypores_lente', onlypores.astype(np.uint8)*255)
            #save the sample mask
            write_sequence(output_folder, 'sample_mask_lente', sample_mask.astype(np.uint8)*255)
            #save the segmented volume
            write_sequence(output_folder, 'binary_lente', binary.astype(np.uint8)*255)

            sheet[f'V{i+2}'] = str(output_folder / 'onlypores_lente')
            sheet[f'U{i+2}'] = str(output_folder / 'sample_mask_lente')
            sheet[f'T{i+2}'] = str(output_folder / 'binary_lente')
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

Processing 1_26
Processing 1_27
Processing 1_28
Processing 1_29
Processing 1_21
Processing 1_22
Processing \\192.168.10.254\NAS2_vol2_Main\11_AIRBUS\03_LabXCT\Panel Pegaso\Probetas\1\22\registered


Progress: 100%|██████████| 319/319 [00:37<00:00,  8.56it/s]


masking
computing otsu
Error processing 1_22
index 0 is out of bounds for axis 0 with size 0
Processing 1_23
Processing \\192.168.10.254\NAS2_vol2_Main\11_AIRBUS\03_LabXCT\Panel Pegaso\Probetas\1\23\registered_lente


Progress: 100%|██████████| 228/228 [00:28<00:00,  8.12it/s]


masking
computing otsu
Error processing 1_23
index 0 is out of bounds for axis 0 with size 0
Processing 1_24
Processing \\192.168.10.254\NAS2_vol2_Main\11_AIRBUS\03_LabXCT\Panel Pegaso\Probetas\1\24\registered


Progress: 100%|██████████| 325/325 [00:37<00:00,  8.75it/s]


masking
computing otsu
Error processing 1_24
index 0 is out of bounds for axis 0 with size 0
Processing 1_31
Processing 1_32
Processing 1_33
Processing 1_35
Processing 1_6
Processing 1_7
Processing 1_8
Processing 1_9
Processing None


In [50]:
wb.save(r'Y:\Database\Database.xlsx')